In [ ]:
# %pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to c:\users\apc\appdata\local\temp\pip-req-build-yuz3ma39
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git 'C:\Users\APC\AppData\Local\Temp\pip-req-build-yuz3ma39'


In [ ]:
# ===== IMPORTS =====
import markdown
from bs4 import BeautifulSoup
import re
from pathlib import Path
from typing import Dict, Any, List
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel
from pymilvus import MilvusClient, DataType, Function, FunctionType, AnnSearchRequest, RRFRanker
from PIL import Image
import torch
from tqdm import tqdm
import clip

In [ ]:
device = "mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [ ]:
# ===== MODEL INITIALIZATION =====
# Text embedding model
text_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2').to(device)

# Multimodal embedding model (CLIP cho text-image joint embedding)
clip_model, clip_preprocess = clip.load("ViT-B/32", device=device)
clip_model.eval()

# Image captioning model (InternVL3.5-2B-Instruct)
# caption_tokenizer = AutoTokenizer.from_pretrained("OpenGVLab/InternVL3_5-2B-Instruct", trust_remote_code=True)
# caption_model = AutoModel.from_pretrained("OpenGVLab/InternVL3_5-2B-Instruct", trust_remote_code=True)

CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          

In [ ]:
print("text_model device:", next(text_model.parameters()).device)
print("clip_model device:", next(clip_model.parameters()).device)

text_model device: cuda:0
clip_model device: cuda:0


In [ ]:
# ===== MARKDOWN PROCESSOR =====
class MarkdownProcessor:
    """Xử lý file markdown, trích xuất text, bảng, và hình ảnh"""

    def __init__(self, markdown_dir: str, image_base_dir: str):
        self.markdown_dir = Path(markdown_dir)
        self.image_base_dir = Path(image_base_dir)

    def extract_content(self, md_file: Path) -> Dict[str, Any]:
        """Trích xuất text, tables, và images từ markdown"""
        with open(md_file, 'r', encoding='utf-8') as f:
            md_content = f.read()

        html = markdown.markdown(md_content, extensions=['tables', 'fenced_code'])
        soup = BeautifulSoup(html, 'html.parser')

        text_content = soup.get_text(separator='\n', strip=True)

        tables = []
        for table in soup.find_all('table'):
            table_text = table.get_text(separator=' | ', strip=True)
            tables.append(table_text)

        image_paths = []
        img_pattern = r'!\[.*?\]\((.*?)\)'
        for match in re.finditer(img_pattern, md_content):
            img_path = match.group(1).strip()
            if img_path.startswith("images/"):
                full_path = self.markdown_dir / img_path
            else:
                full_path = self.image_base_dir / img_path

            if full_path.exists():
                image_paths.append(str(full_path))

        return {
            'text': text_content,
            'tables': tables,
            'images': image_paths,
            'source': str(md_file)
        }

In [ ]:
class MultimodalImageEncoder:
    """Encode images and text with CLIP embeddings"""

    def __init__(self, clip_model, clip_preprocess):
        self.clip_model = clip_model
        self.clip_preprocess = clip_preprocess
        self.device = "mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu"

        # Move model to device
        self.clip_model = self.clip_model.to(self.device)

    def encode_image_multimodal(self, image_path: str) -> List[float]:
        """Encode image into CLIP embedding vector"""
        try:
            image = Image.open(image_path).convert('RGB')
            image_input = self.clip_preprocess(image).unsqueeze(0).to(self.device)

            with torch.no_grad():
                image_features = self.clip_model.encode_image(image_input)
                image_features /= image_features.norm(dim=-1, keepdim=True)

            return image_features.squeeze().cpu().numpy().tolist()
        except Exception as e:
            print(f"Error encoding image {image_path}: {e}")
            # Return zero vector of appropriate dimension
            return [0.0] * 512  # CLIP's standard dimension

    def encode_text_for_image_search(self, text: str) -> List[float]:
        """Encode text into CLIP embedding vector for text-to-image search"""
        try:
            text_tokens = clip.tokenize(text).to(self.device)

            with torch.no_grad():
                text_features = self.clip_model.encode_text(text_tokens)
                text_features /= text_features.norm(dim=-1, keepdim=True)

            return text_features.squeeze().cpu().numpy().tolist()
        except Exception as e:
            print(f"Error encoding text '{text}': {e}")
            # Return zero vector of appropriate dimension
            return [0.0] * 512  # CLIP's standard dimension

image_encoder = MultimodalImageEncoder(clip_model, clip_preprocess)

In [ ]:
# ===== ENTITY PROCESSORS =====
class TextEntityProcessor:
    """Xử lý entity text riêng biệt"""

    def __init__(self, text_model, chunk_size=500, overlap=100):
        self.text_model = text_model
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=overlap
        )

    def create_text_entities(self, content: Dict[str, Any]) -> List[Dict[str, Any]]:
        """Tạo text entities"""
        full_text = content['text']

        # Thêm tables vào text nếu có
        if content['tables']:
            full_text += "\n\n" + "\n\n".join(content['tables'])

        text_chunks = self.text_splitter.create_documents([full_text])

        entities = []
        for idx, chunk in enumerate(text_chunks):
            # Text embedding
            text_embedding = self.text_model.encode(chunk.page_content, convert_to_tensor=False).tolist()

            entity = {
                # 'id': f"text_{idx}",
                'content': chunk.page_content,
                'text_dense': text_embedding,  # Dense vector cho semantic search
                'text_sparse': chunk.page_content,  # Text cho BM25 sparse search
                'metadata': {
                    'entity_type': 'text',
                    'source': content['source'],
                    'chunk_index': idx,
                    'content_type': 'text_with_tables' if content['tables'] else 'text_only'
                },
                # Các trường image để trống
                'image_path': '',
                # 'image_caption': '',
                'image_dense': [0.0] * 512  # Vector trống với đúng dimension
            }
            entities.append(entity)

        return entities

In [ ]:
class ImageEntityProcessor:
    """Xử lý entity image riêng biệt"""

    def __init__(self, image_encoder):
        self.image_encoder = image_encoder

    def create_image_entities(self, content: Dict[str, Any]) -> List[Dict[str, Any]]:
        """Tạo image entities"""
        entities = []

        for idx, img_path in enumerate(content['images']):
            # Generate caption
            # caption = self.image_encoder.generate_caption(img_path)

            # Multimodal embedding
            image_embedding = self.image_encoder.encode_image_multimodal(img_path)

            # Text embedding của caption (cho text search)
            # caption_embedding = text_model.encode(caption, convert_to_tensor=False).tolist()

            entity = {
                # 'id': f"image_{idx}",
                'content': "",  # Caption làm content chính
                'image_path': img_path,
                # 'image_caption': caption,
                'image_dense': image_embedding,  # Multimodal vector cho image search
                'metadata': {
                    'entity_type': 'image',
                    'source': content['source'],
                    'image_index': idx,
                    'original_path': img_path
                },
                # Các trường text bỏ trống cho image
                'text_dense': [0.0] * 768,  # Dense vector của caption
                # 'text_sparse': caption  # Caption text cho BM25
            }
            entities.append(entity)

        return entities

In [ ]:
text_processor = TextEntityProcessor(text_model)
image_processor = ImageEntityProcessor(image_encoder)

In [ ]:
# ===== MILVUS HYBRID STORE =====
class MilvusHybridStore:
    """Milvus store tối ưu cho hybrid search"""

    def __init__(self, uri, token, collection_name="public"):
        self.client = MilvusClient(uri=uri, token=token)
        self.collection_name = collection_name
        self.text_dense_dim = 768  # paraphrase-multilingual-mpnet-base-v2
        self.image_dense_dim = 512  # clip

    def create_hybrid_collection(self):
        """Tạo collection cho hybrid search"""
        # if self.client.has_collection(self.collection_name):
        #     self.client.drop_collection(self.collection_name)

        schema = self.client.create_schema(
            auto_id=True,
            enable_dynamic_fields=True
        )

        # Primary key
        schema.add_field(
            field_name="id",
            datatype=DataType.VARCHAR,
            is_primary=True,
            max_length=100
        )

        # Content field
        schema.add_field(
            field_name="content",
            datatype=DataType.VARCHAR,
            max_length=8000,
            enable_analyzer=True,
        )

        # Text fields
        schema.add_field(
            field_name="text_dense",
            datatype=DataType.FLOAT_VECTOR,
            dim=self.text_dense_dim,
        )

        schema.add_field(
            field_name="text_sparse",
            datatype=DataType.SPARSE_FLOAT_VECTOR,
        )

        # Image fields
        schema.add_field(
            field_name="image_dense",
            datatype=DataType.FLOAT_VECTOR,
            dim=self.image_dense_dim,
        )

        schema.add_field(
            field_name="image_path",
            datatype=DataType.VARCHAR,
            max_length=500,
        )

        # schema.add_field(
        #     field_name="image_caption",
        #     datatype=DataType.VARCHAR,
        #     max_length=1000,
        # )

        # Metadata
        schema.add_field(
            field_name="metadata",
            datatype=DataType.JSON
        )

        # BM25 function cho text sparse
        bm25_function = Function(
            name="text_bm25_emb",
            function_type=FunctionType.BM25,
            input_field_names=["content"],
            output_field_names=["text_sparse"]
        )
        schema.add_function(bm25_function)

        # Index parameters
        index_params = self.client.prepare_index_params()

        # Text dense index
        index_params.add_index(
            field_name="text_dense",
            index_type="HNSW",
            metric_type="COSINE",
            params={"M": 16, "efConstruction": 200}
        )

        # Text sparse index
        index_params.add_index(
            field_name="text_sparse",
            index_type="SPARSE_INVERTED_INDEX",
            metric_type="BM25"
        )

        # Image dense index
        index_params.add_index(
            field_name="image_dense",
            index_type="HNSW",
            metric_type="COSINE",
            params={"M": 16, "efConstruction": 200}
        )

        self.client.create_collection(
            collection_name=self.collection_name,
            schema=schema,
            index_params=index_params
        )

        print(f"✅ Hybrid collection '{self.collection_name}' đã được tạo!")

    def insert_entities(self, entities: List[Dict[str, Any]]):
        """Insert entities vào collection"""
        batch_size = 50
        for i in range(0, len(entities), batch_size):
            batch = entities[i:i + batch_size]

            # Remove text_sparse from entities as it's generated by BM25 function
            for entity in batch:
                entity.pop('text_sparse', None)

            self.client.insert(
                collection_name=self.collection_name,
                data=batch
            )
            print(f"✅ Đã insert batch {i//batch_size + 1}/{(len(entities)-1)//batch_size + 1}")

        print(f"✅ Đã insert {len(entities)} entities vào Milvus!")

In [ ]:
from pymilvus import AnnSearchRequest, RRFRanker

class HybridSearchEngine:
    """Engine cho hybrid search với multiple vectors từ text input"""

    def __init__(self, milvus_store, text_model, image_encoder):
        self.store = milvus_store
        self.text_model = text_model
        self.image_encoder = image_encoder

    def hybrid_search(self, query_text: str, limit: int = 10) -> List[Dict]:
        """Thực hiện hybrid search chỉ với text input"""

        search_requests = []

        if not query_text:
            return []

        # 1. Text semantic search với text embedding model
        query_text_embedding = self.text_model.encode(query_text, convert_to_tensor=True, device=device).tolist()

        text_search = AnnSearchRequest(
            data=[query_text_embedding],
            anns_field="text_dense",
            param={"nprobe": 10},
            limit=limit
        )
        search_requests.append(text_search)

        # 2. Full-text search (BM25) với sparse vectors
        sparse_search = AnnSearchRequest(
            data=[query_text],
            anns_field="text_sparse",
            param={"drop_ratio_search": 0.2},
            limit=limit
        )
        search_requests.append(sparse_search)

        # 3. Multimodal search - embedding text bằng image encoder
        query_image_embedding = self.image_encoder.encode_text_for_image_search(query_text)

        image_search = AnnSearchRequest(
            data=[query_image_embedding],
            anns_field="image_dense",
            param={"nprobe": 10},
            limit=limit
        )
        search_requests.append(image_search)

        # Thực hiện hybrid search với RRF ranker
        ranker = RRFRanker(100)

        results = self.store.client.hybrid_search(
            collection_name=self.store.collection_name,
            reqs=search_requests,
            ranker=ranker,
            limit=limit,
            output_fields=["content", "metadata", "image_path"]
        )

        return results[0] if results else []

In [ ]:
# ===== MAIN PIPELINE =====
def create_hybrid_pipeline(markdown_dir: str, image_base_dir: str,
                            uri: str, token: str):
    """Tạo pipeline hybrid search hoàn chỉnh"""

    # Initialize components
    processor = MarkdownProcessor(markdown_dir, image_base_dir)
    store = MilvusHybridStore(uri, token)
    search_engine = HybridSearchEngine(store, text_model, image_encoder)

    # Tạo collection
    store.create_hybrid_collection()

    def process_documents(markdown_files: List[str]):
        """Xử lý documents thành entities"""
        all_entities = []

        for md_file in markdown_files:
            print(f"\n{'='*60}")
            print(f"Đang xử lý: {md_file}")
            print(f"{'='*60}")

            # Extract content
            content = processor.extract_content(Path(md_file))
            print(f"  - Text length: {len(content['text'])} ký tự")
            print(f"  - Số bảng: {len(content['tables'])}")
            print(f"  - Số hình ảnh: {len(content['images'])}")

            # Tạo text entities
            text_entities = text_processor.create_text_entities(content)
            print(f"  - Text entities: {len(text_entities)}")

            # Tạo image entities
            image_entities = image_processor.create_image_entities(content)
            print(f"  - Image entities: {len(image_entities)}")

            all_entities.extend(text_entities)
            all_entities.extend(image_entities)

                # Insert vào Milvus
        store.insert_entities(all_entities)

        print(f"\n✅ Pipeline hoàn tất! Tổng cộng {len(all_entities)} entities đã được xử lý.")
        return all_entities, search_engine

    return process_documents, search_engine

In [ ]:
import os
PATH = "data/training_input/"
file_names = [f for f in os.listdir(PATH) if f.lower().endswith(".pdf")]
file_names = [f[:-4] for f in file_names]
file_names

['Public001',
 'Public002',
 'Public003',
 'Public004',
 'Public005',
 'Public006',
 'Public007',
 'Public008',
 'Public009',
 'Public010',
 'Public011',
 'Public012',
 'Public013',
 'Public014',
 'Public015',
 'Public016',
 'Public017',
 'Public018',
 'Public019',
 'Public020',
 'Public021',
 'Public022',
 'Public023',
 'Public024',
 'Public025',
 'Public026',
 'Public027',
 'Public028',
 'Public029',
 'Public030',
 'Public031',
 'Public032',
 'Public033',
 'Public034',
 'Public035',
 'Public036',
 'Public037',
 'Public038',
 'Public039',
 'Public040',
 'Public041',
 'Public042',
 'Public043',
 'Public044',
 'Public045',
 'Public046',
 'Public047',
 'Public048',
 'Public049',
 'Public050',
 'Public051',
 'Public052',
 'Public053',
 'Public054',
 'Public055',
 'Public056',
 'Public057',
 'Public058',
 'Public059',
 'Public060']

In [ ]:
# Configuration
# IMAGE_BASE_DIR = 'test_data/images'
# MILVUS_URI = "http://localhost:19530"
# MILVUS_TOKEN = "root:Milvus"

In [ ]:
# Configuration
IMAGE_BASE_DIR = 'test_data/images'
MILVUS_URI = "https://in03-7b3b56e59d62e9d.serverless.aws-eu-central-1.cloud.zilliz.com"
MILVUS_TOKEN = "30cff684b802d87f26e0c7ea80e43c759237808981ac1563ae400b00316ff84be4261492ee91b9f55ec6ad8a25b7be9b483fc957"

In [ ]:
# for file_name in file_names:
#     MARKDOWN_DIR = f'out\\{file_name}'
#     process_docs, search_engine = create_hybrid_pipeline(
#         MARKDOWN_DIR, IMAGE_BASE_DIR, MILVUS_URI, MILVUS_TOKEN
#     )

#     # Xử lý documents
#     markdown_files = list(Path(MARKDOWN_DIR).glob("*.md"))
#     print(markdown_files)
#     entities, search_engine = process_docs(markdown_files) # type: ignore

In [ ]:
from pprint import pprint

store = MilvusHybridStore(MILVUS_URI, MILVUS_TOKEN)
search_engine = HybridSearchEngine(store, text_model, image_encoder)

In [ ]:
print(f"\n{'='*60}")
print("TEST HYBRID SEARCH")
print(f"{'='*60}")

# Text search
text_results = search_engine.hybrid_search(
    query_text="Trong mô hình nhà thông minh, IoT chủ yếu đóng vai trò gì?",
    limit=5
)
print(f"Text search results: {len(text_results)}")
for i, result in enumerate(text_results[:3]):
    pprint(f"  {i+1}. Score: {result['distance']:.4f}")
    pprint(f"     Content: {result['entity']['content']}...")
    pprint(f"     Type: {result['entity']['metadata']['entity_type']}")


TEST HYBRID SEARCH
Text search results: 5
'  1. Score: 0.0195'
('     Content: Công nghệ ứng dụng vào mô hình nhà thông minh dựa trên '
 'Internet vạn vật (IoT - Internet of Things) phần lớn bị hạn chế và phân tán. '
 'Các đánh giá trong bài viết được thực hiện để phân loại bối cảnh nghiên cứu '
 'về ứng dụng IoT xây dựng mô hình nhà thông minh, nhằm cung cấp những hiểu '
 'biết có giá trị về công nghệ và hỗ trợ các nhà nghiên cứu hiểu các nền tảng '
 'có sẵn và các lỗ hổng trong lĩnh vực này. Chúng tôi tiến hành tìm kiếm các '
 'bài viết liên quan đến (1) nhà thông minh, (2) ứng dụng và (3) IoT...')
'     Type: text'
'  2. Score: 0.0192'
('     Content: 1.5 Nhà thông minh từ sau năm 2010\n'
 'Trong phần này sẽ tập trung vào sự tích hợp của nhà thông minh, IoT và điện '
 'toán đám mây để xác định một mô hình điện toán mới. Có thể tìm thấy trong '
 'phần tài liệu các cuộc khảo sát và nghiên cứu về nhà thông minh, IoT và điện '
 'toán đám mây, các thuộc tính, tính năng, công nghệ và như

## LLM with LangChain

In [ ]:
import torch
torch.cuda.memory_allocated() / 1e9  # check memory in GB

3.548063744

## Full Chain

In [ ]:
# from langchain.prompts import PromptTemplate
# from langchain_core.output_parsers import PydanticOutputParser
# import re

# # --- Define Output Model ---
# from pydantic import BaseModel

# class MCQAnswer(BaseModel):
#     answer: str
#     explanation: str

# class MCQInput(BaseModel):
#     question: str
#     option_a: str
#     option_b: str
#     option_c: str
#     option_d: str

# # --- Define Prompt ---
# parser = PydanticOutputParser(pydantic_object=MCQAnswer)

# prompt = PromptTemplate(
#     template=(
#         "You are a strict assistant. "
#         "Use the given context to answer the multiple-choice question below.\n\n"
#         "Return only a JSON object in this format:\n"
#         "{{\"answer\": \"<A|B|C|D>\", \"explanation\": \"<short reason>\"}}\n\n"
#         "Context: {context}\n"
#         "Question: {question}\n"
#         "A. {option_a}\nB. {option_b}\nC. {option_c}\nD. {option_d}\n\n"
#         "Return *only* a JSON object matching this schema:\n"
#         "{format_instructions}\n"
#         "Do not add any extra text.\n"
#     ),
#     input_variables=["context", "question", "option_a", "option_b", "option_c", "option_d"],
#     partial_variables={"format_instructions": parser.get_format_instructions()},
# )

# # --- Create Chain ---
# full_chain = prompt | llm | parser

### Demo Response

In [ ]:
# # --- Input ---
# mcq = MCQInput(
#     question="Trong mô hình nhà thông minh, IoT chủ yếu đóng vai trò gì?",
#     option_a="Lưu trữ dữ liệu trên máy chủ",
#     option_b="Kết nối Internet và quản lý thiết bị từ xa",
#     option_c="Cung cấp dịch vụ phân tích dữ liệu lớn",
#     option_d="Thay thế hoàn toàn điện toán đám mây",
# )

# retrieved_context = get_relevant_documents("Trong mô hình nhà thông minh, IoT chủ yếu đóng vai trò gì ?")
# response = llm.invoke(prompt.format(**mcq.model_dump(), context=retrieved_context))
# print("Raw response:", response)  # Check if it's a dict with 'text' key

Raw response: You are a strict assistant. Use the given context to answer the multiple-choice question below.

Return only a JSON object in this format:
{"answer": "<A|B|C|D>", "explanation": "<short reason>"}

Context: [Document(metadata={'entity_type': 'text', 'source': 'out\\Public001\\main.md', 'chunk_index': 0, 'content_type': 'text_only'}, page_content='Công nghệ ứng dụng vào mô hình nhà thông minh dựa trên Internet vạn vật (IoT - Internet of Things) phần lớn bị hạn chế và phân tán. Các đánh giá trong bài viết được thực hiện để phân loại bối cảnh nghiên cứu về ứng dụng IoT xây dựng mô hình nhà thông minh, nhằm cung cấp những hiểu biết có giá trị về công nghệ và hỗ trợ các nhà nghiên cứu hiểu các nền tảng có sẵn và các lỗ hổng trong lĩnh vực này. Chúng tôi tiến hành tìm kiếm các bài viết liên quan đến (1) nhà thông minh, (2) ứng dụng và (3) IoT'), Document(metadata={'entity_type': 'text', 'source': 'out\\Public001\\main.md', 'chunk_index': 4, 'content_type': 'text_only'}, page_con

In [ ]:

# #? Extract answer and explanation using regex
# pattern = r'\{"answer":\s*"([A-D])",\s*"explanation":\s*"([^"]+)"\}'
# match = re.search(pattern, response)

# if match:
#     answer = match.group(1)
#     explanation = match.group(2)

#     print(f"Answer: {answer}")
#     print(f"Explanation: {explanation}")
# else:
#     print("No match found")

Answer: B
Explanation: IoT trong mô hình nhà thông minh chủ yếu giúp kết nối Internet và quản lý từ xa các thiết bị di động, được tích hợp với nhiều loại cảm biến.


# Inference 

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoModelForImageTextToText, AutoTokenizer, BitsAndBytesConfig, pipeline
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoProcessor, AutoModelForVision2Seq


def load_llm():
    MODEL_PATH = "Qwen/Qwen2.5-3B-Instruct"  # Local model path

    # ✅ Optimized 4-bit quantization config for Blackwell GPUs
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,     # Nested quantization → less VRAM
        bnb_4bit_quant_type="nf4",          # Best quantization format for LLMs
        bnb_4bit_compute_dtype=torch.bfloat16,  # BF16 is optimal on Blackwell
    )

    # ✅ Load tokenizer & model locally with full GPU optimization
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_PATH,
        quantization_config=bnb_config,
        device_map="auto",                  # Automatically spreads across GPUs if available
        dtype=torch.bfloat16,         # Native dtype for new NVIDIA architectures
        local_files_only=True,
    )

    # ✅ Enable better CUDA performance
    torch.backends.cuda.matmul.allow_tf32 = True     # TensorFloat-32 acceleration
    torch.backends.cudnn.benchmark = True            # Optimize kernel selection
    torch.set_float32_matmul_precision("high")       # Prefer high precision kernels

    # ✅ Create high-throughput inference pipeline
    generation_pipeline = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=256,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.1,
    )

    return HuggingFacePipeline(pipeline=generation_pipeline)

llm = load_llm()
print("Model device:", next(llm.pipeline.model.parameters()).device)

Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.00s/it]
Device set to use cuda:0


Model device: cuda:0


In [ ]:
import base64
from io import BytesIO
from langchain.chains import RetrievalQA
from langchain.schema import Document
"""
{
    "id": null,
    "metadata": {
        "entity_type": "text",
        "source": "out\\Public001\\main.md",
        "chunk_index": 0,
        "content_type": "text_only"
    },
    "page_content": "Công nghệ ứng dụng vào mô hình nhà thông minh dựa trên Internet vạn vật (IoT - Internet of Things) phần lớn bị hạn chế và phân tán. Các đánh giá trong bài viết được thực hiện để phân loại bối cảnh nghiên cứu về ứng dụng IoT xây dựng mô hình nhà thông minh, nhằm cung cấp những hiểu biết có giá trị về công nghệ và hỗ trợ các nhà nghiên cứu hiểu các nền tảng có sẵn và các lỗ hổng trong lĩnh vực này. Chúng tôi tiến hành tìm kiếm các bài viết liên quan đến (1) nhà thông minh, (2) ứng dụng và (3) IoT",
    "type": "Document"
},
"""

# def get_relevant_documents(query: str) -> list[Document]:
#     results = search_engine.hybrid_search(query, limit=5)
#     docs = []
#     for result in results:
#         content = result['entity']['content']
#         if result['entity']['image_path']:
#             content += f" [Image: {result['entity']['image_path']}]"

#         docs.append(Document(page_content=content, metadata=result['entity']['metadata']))

#     return docs

def get_relevant_documents(query: str) -> list[Document]:
    results = search_engine.hybrid_search(query, limit=5)
    docs = []
    for result in results:
        content = result['entity']['content']
        if result['entity']['image_path']:
            # Đọc và mã hóa hình ảnh thành base64 để nhúng vào context
            try:
                image_path = result['entity']['image_path']
                with Image.open(image_path) as img:
                    buffered = BytesIO()
                    img.save(buffered, format="PNG")
                    img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
                    content += f" <image>{img_str}</image>"  # Định dạng phù hợp với Qwen-VL
            except Exception as e:
                content += f" [Failed to load image: {str(e)}]"

        docs.append(Document(page_content=content, metadata=result['entity']['metadata']))

    return docs

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
import re
import pandas as pd
from pydantic import BaseModel


csv_path = "question.csv"  # Update if needed
# Read the CSV into a DataFrame
df = pd.read_csv(csv_path)


class MCQAnswer(BaseModel):
    answer: str

class MCQInput(BaseModel):
    question: str
    option_a: str
    option_b: str
    option_c: str
    option_d: str


# --- Define Prompt ---
parser = PydanticOutputParser(pydantic_object=MCQAnswer)

prompt = PromptTemplate(
    template=(
        "You are a strict assistant. "
        "Use the given context to answer the multiple-choice question below.\n\n"
        "Return only a JSON object in this format:\n"
        "{{\"answer\": \"<A|B|C|D>\"}}\n\n"
        "Context: {context}\n"
        "Question: {question}\n"
        "A. {option_a}\nB. {option_b}\nC. {option_c}\nD. {option_d}\n\n"
        "Return the correct answer as letter (i.e. A, B, C, D). If there are multiple correct answer then seperate them by a comma e.g. 'A, B'"
        "Do not add any extra text.\n"
    ),
    input_variables=["context", "question", "option_a", "option_b", "option_c", "option_d"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# --- Create Chain ---
full_chain = prompt | llm | parser

In [ ]:

# --- Input ---
mcq = MCQInput(
    question="Trong mô hình nhà thông minh, IoT chủ yếu đóng vai trò gì?",
    option_a="Lưu trữ dữ liệu trên máy chủ",
    option_b="Kết nối Internet và quản lý thiết bị từ xa",
    option_c="Cung cấp dịch vụ phân tích dữ liệu lớn",
    option_d="Thay thế hoàn toàn điện toán đám mây",
)

context = "IoT trong nhà thông minh giúp kết nối, điều khiển và quản lý các thiết bị qua mạng Internet."
response = llm.invoke(prompt.format(**mcq.model_dump(), context=context))
print("Raw response:", response)

# Extract answer using regex
pattern = r'\{"answer":\s*"([A-D, ]+)"\}'
match = re.search(pattern, response)

if match:
    answer = match.group(1)
    print(f"{answer}")
else:
    print("No match found")

Raw response: You are a strict assistant. Use the given context to answer the multiple-choice question below.

Return only a JSON object in this format:
{"answer": "<A|B|C|D>"}

Context: IoT trong nhà thông minh giúp kết nối, điều khiển và quản lý các thiết bị qua mạng Internet.
Question: Trong mô hình nhà thông minh, IoT chủ yếu đóng vai trò gì?
A. Lưu trữ dữ liệu trên máy chủ
B. Kết nối Internet và quản lý thiết bị từ xa
C. Cung cấp dịch vụ phân tích dữ liệu lớn
D. Thay thế hoàn toàn điện toán đám mây

Return the correct answer as letter (i.e. A, B, C, D). If there are multiple correct answer then seperate them by a comma e.g. 'A, B'Do not add any extra text.
{"answer": "B"}
B


In [ ]:
# --- Input ---
# Extract each row into a list of MCQInput objects
mcq_list = []
for _, row in df.iterrows():
    mcq = MCQInput(
        question=str(row['Question']) if pd.notna(row['Question']) else "",
        option_a=str(row['A']) if pd.notna(row['A']) else "",
        option_b=str(row['B']) if pd.notna(row['B']) else "",
        option_c=str(row['C']) if pd.notna(row['C']) else "",
        option_d=str(row['D']) if pd.notna(row['D']) else ""
    )
    mcq_list.append(mcq)

# Now mcq_list contains all MCQInput objects
print(f"Total questions loaded: {len(mcq_list)}")


answers = []
# Example: Process the first one (you can loop over all)
for i, mcq in enumerate(mcq_list):  # Process first 5 for demo
    print(f"Processing question {i+1}: {mcq.question}")
    # Your existing code for retrieval and inference
    retrieved_context = get_relevant_documents(mcq.question)
    context = "\n".join([doc.page_content for doc in retrieved_context])
    response = llm.invoke(prompt.format(context=context, **mcq.model_dump()))
    # Extract answer using regex
    pattern = r'\{"answer":\s*"([A-D, ]+)"\}'
    match = re.search(pattern, response)

    if match:
        answer = match.group(1)
        print(f"{answer}")
        answers.append(answer)
    else:
        print("No match found")
        answers.append('C')


Total questions loaded: 300
Processing question 1: Trong mô hình nhà thông minh, IoT chủ yếu đóng vai trò gì?
B
Processing question 2: Năm 2010, công nghệ nào thường được sử dụng trong kiểm soát ra vào ngôi nhà thông minh?
B
Processing question 3: Bộ cảm biến trong nhà thông minh thực hiện chức năng gì và bộ truyền động có vai trò gì?
C
Processing question 4: Nội dung chính của bài báo nghiên cứu về IoT trong xây dựng nhà thông minh là gì?
C
Processing question 5: Trong nghiên cứu, 229 bài báo được chia thành 4 lớp. Trung bình mỗi lớp có bao nhiêu bài?
B
Processing question 6: Học phần Bảo mật Web trang bị kiến thức về bộ rủi ro phổ biến nào?
A
Processing question 7: Trong học phần Cơ sở dữ liệu nâng cao, loại CSDL nào sau đây không được đề cập?
D
Processing question 8: Học phần bảo mật web, csdl dạy nội dung gì?
A
Processing question 9: Nhóm môn học trong tài liệu Public002 tập trung cung cấp cho sinh viên kiến thức tổng quan về lĩnh vực nào?
Error encoding text 'Nhóm môn học trong tà

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


C
Processing question 12: Công cụ nào được nhóm tác giả sử dụng để huấn luyện các hệ dịch nơ ron?
B
Processing question 13: Điểm BLEU và công cụ huấn luyện hệ dịch nơ ron trong nghiên cứu này là gì?
D
Processing question 14: Nội dung chính của bài báo Public003 là gì?
B
Processing question 15: Theo kết quả thử nghiệm, hệ dịch Adapt_System cải thiện bao nhiêu điểm BLEU so với Baseline_G trong miền pháp lý?
Error encoding text 'Theo kết quả thử nghiệm, hệ dịch Adapt_System cải thiện bao nhiêu điểm BLEU so với Baseline_G trong miền pháp lý?': Input Theo kết quả thử nghiệm, hệ dịch Adapt_System cải thiện bao nhiêu điểm BLEU so với Baseline_G trong miền pháp lý? is too long for context length 77
D
Processing question 16: Công nghệ in bê tông 3D được thực hiện theo quy trình nào?
C
Processing question 17: Công nghệ nào được phát triển tại Đại học Loughborough (Anh)?
B
Processing question 18: Công nghệ in bê tông 3D nào vừa cho phép kiểm soát tốt cấu trúc sản phẩm, vừa sử dụng bê tông cốt liệ

In [46]:
11520 - 11221

299

In [43]:
answers

['B',
 'B',
 'C',
 'C',
 'B',
 'A',
 'D',
 'A',
 'B',
 'D',
 'C',
 'B',
 'D',
 'B',
 'D',
 'C',
 'B',
 'B',
 'C',
 'A',
 'A',
 'B',
 'B',
 'D',
 'C',
 'B',
 'A',
 'B',
 'D',
 'D',
 'B',
 'C',
 'B',
 'B',
 'A',
 'B',
 'C',
 'A',
 'B',
 'B',
 'A',
 'C',
 'C',
 'C',
 'C',
 'C',
 'A',
 'A',
 'C',
 'D',
 'C',
 'B',
 'A',
 'D',
 'A',
 'B',
 'C',
 'A',
 'D',
 'C',
 'A',
 'B',
 'A',
 'B',
 'C',
 'A',
 'C',
 'B',
 'D',
 'C',
 'B',
 'A',
 'B',
 'D',
 'C',
 'B',
 'A',
 'C',
 'C',
 'C',
 'B',
 'A',
 'A',
 'B',
 'C',
 'D',
 'C',
 'A',
 'B',
 'C',
 'D',
 'B',
 'B',
 'D',
 'C',
 'B',
 'C',
 'C',
 'C',
 'C',
 'B',
 'C',
 'B',
 'C',
 'A',
 'C',
 'C',
 'B',
 'C',
 'B',
 'C',
 'A',
 'A',
 'D',
 'B',
 'C',
 'B',
 'A',
 'C',
 'B',
 'C',
 'D',
 'A',
 'C',
 'C',
 'C',
 'D',
 'D',
 'C',
 'B',
 'C',
 'C',
 'D',
 'C',
 'C',
 'B',
 'A',
 'B',
 'A',
 'A',
 'B',
 'A',
 'B',
 'C',
 'A',
 'B',
 'A',
 'D',
 'A',
 'B',
 'B',
 'A',
 'C',
 'A',
 'B',
 'B',
 'B',
 'A',
 'B',
 'C',
 'B',
 'D',
 'C',
 'A',
 'C',
 'A',
 'B'

In [ ]:
# Save to CSV
output_path = "answers.csv"
output_df.to_csv(answers, index=False)

ValueError: Length of values (294) does not match length of index (300)

## VLM

In [ ]:
# import torch
# from transformers import AutoModelForCausalLM, AutoModelForImageTextToText, AutoTokenizer, BitsAndBytesConfig, pipeline
# from langchain_huggingface import HuggingFacePipeline
# from transformers import AutoProcessor



# def load_llm():
#     # MODEL_PATH = "Qwen/Qwen2.5-3B-Instruct"  # Local model path
#     MODEL_PATH = 'Qwen/Qwen2.5-VL-3B-Instruct'

#     # ✅ Optimized 4-bit quantization config for Blackwell GPUs
#     bnb_config = BitsAndBytesConfig(
#         load_in_4bit=True,
#         bnb_4bit_use_double_quant=True,     # Nested quantization → less VRAM
#         bnb_4bit_quant_type="nf4",          # Best quantization format for LLMs
#         bnb_4bit_compute_dtype=torch.bfloat16,  # BF16 is optimal on Blackwell
#     )

#     # ✅ Load tokenizer & model locally with full GPU optimization
#     processor = AutoProcessor.from_pretrained(MODEL_PATH, trust_remote_code=True)
#     model = AutoModelForImageTextToText.from_pretrained(
#         MODEL_PATH,
#         quantization_config=bnb_config,
#         device_map="auto",                  # Automatically spreads across GPUs if available
#         torch_dtype=torch.bfloat16,         # Native dtype for new NVIDIA architectures
#         trust_remote_code=True,
#         local_files_only=True,
#     )

#     # model = AutoModelForCausalLM.from_pretrained(
#     #     MODEL_PATH,
#     #     quantization_config=bnb_config,
#     #     device_map="auto",                  # Automatically spreads across GPUs if available
#     #     dtype=torch.bfloat16,         # Native dtype for new NVIDIA architectures
#     #     local_files_only=True,
#     # )

#     # ✅ Enable better CUDA performance
#     torch.backends.cuda.matmul.allow_tf32 = True     # TensorFloat-32 acceleration
#     torch.backends.cudnn.benchmark = True            # Optimize kernel selection
#     torch.set_float32_matmul_precision("high")       # Prefer high precision kernels

#     # ✅ Create high-throughput inference pipeline
#     generation_pipeline = pipeline(
#         "image-text-to-text",
#         model=model,
#         tokenizer=processor.tokenizer,
#         max_new_tokens=256,
#         # temperature=0.7,
#         # top_p=0.9,
#         # repetition_penalty=1.1,
#     )

#     return HuggingFacePipeline(pipeline=generation_pipeline)


# llm = load_llm()
# print("Model device:", next(llm.pipeline.model.parameters()).device)
